# Route optimization of a historical sites with ORS and `ortools`
> Note: All notebooks need the [environment dependencies](https://github.com/GIScience/openrouteservice-examples#local-installation)
> as well as an [openrouteservice API key](https://openrouteservice.org/dev/#/signup) to run

In [1]:
import folium

In [2]:
api_key = 'API_KEY'
aoi_centroid = (50.062208,19.936838)  # Kraków center for map center

Next, add the Kreuzberg polygon as marker to the map, so we get a bit of orientation.

In [3]:
m = folium.Map(location=aoi_centroid, zoom_start=16)

m

We use the [**Places API**](https://openrouteservice.org/dev/#/api-docs/pois),
where we can pass a GeoJSON as object right into.

In [4]:
from openrouteservice import client, places

ors = client.Client(key=api_key)

[**Here**](https://giscience.github.io/openrouteservice/documentation/Places.html) is a nicer list.

In [5]:
# bounding_box = [[19.960098266601562,50.04468336057252],[19.92568016052246,50.067828014342446]]
# geo_json = {"type":"Point","coordinates":[19.939756393432617,50.060830689417585]}
#
# geometr = {
#
# }
# query = {'request': 'pois',
#          'geojson': geo_json,
#          'bbox': bounding_box,
#          # 'filter_category_group_ids': [220, 330],
#          # 'sortby': 'distance',
#          'limit':10}
# ors.places(**query)


import requests
import json

body = {"request":"pois","geometry":{"buffer":200,"bbox":[[19.960098266601562,50.04468336057252],[19.92568016052246,50.067828014342446]],"geojson":{"type":"Point","coordinates":[19.939756393432617,50.060830689417585]}},"limit":10}

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': '5b3ce3597851110001cf6248f11c1c7ad0cc46a9ba5e768b7977ed3e',
    'Content-Type': 'application/json; charset=utf-8'
}
call = requests.post('https://api.openrouteservice.org/pois', json=body, headers=headers)
features = json.loads(call.text)

In [6]:

sites = features['features']  # Perform the actual request and get inner json

print("\nCount of sites: {}".format(len(sites)))
sites


Count of sites: 9


[{'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [19.937343, 50.061671]},
  'properties': {'osm_id': 23256528,
   'osm_type': 2,
   'distance': 196.428883,
   'category_ids': {'622': {'category_name': 'attraction',
     'category_group': 'tourism'}},
   'osm_tags': {'name': 'Sukiennice'}}},
 {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [19.939471, 50.06164]},
  'properties': {'osm_id': 26195267,
   'osm_type': 2,
   'distance': 92.27702311,
   'category_ids': {'622': {'category_name': 'attraction',
     'category_group': 'tourism'},
    '135': {'category_name': 'place_of_worship',
     'category_group': 'arts_and_culture'}},
   'osm_tags': {'name': 'Kościół pw. Wniebowzięcia Najświętszej Marii Panny',
    'website': 'http://www.mariacki.com/',
    'opening_hours': 'Mo-Sa 11:30-18:00; Su 14:00-18:00'}}},
 {'type': 'Feature',
  'geometry': {'type': 'Point', 'coordinates': [19.937767, 50.060895]},
  'properties': {'osm_id': 29271113,
   'osm_type':

In [7]:
sites_addresses = []

for feat in sites:
    lon, lat = feat['geometry']['coordinates']
    name = feat['properties']['osm_tags']['name'] if 'name' in feat['properties']['osm_tags'] else ''
    popup = "<strong>{0}</strong><br>Lat: {1:.3f}<br>Long: {2:.3f}".format(name, lat, lon)
    icon = folium.map.Icon(color='lightgray',
                           icon_color='#b5231a',
                           icon='beer',  # fetches font-awesome.io symbols
                           prefix='fa')
    folium.map.Marker([lat, lon], popup=popup).add_to(m)
    sites_addresses.append(name)

# folium.map.LayerControl().add_to(m)
m

Ok, we have an idea where we go.
But, not in which order.
To determine the optimal route, we first have to know the distance between all sites.
We can conveniently solve this with the [**Matrix API**](https://openrouteservice.org/dev/#/api-docs/matrix).
> I'd have like to do this example for biking/walking, but I realized too late that we restricted matrix calls to 5x5 locations for those profiles...

In [8]:
from openrouteservice import distance_matrix

sites_coords = [feat['geometry']['coordinates'] for feat in sites]

request = {'locations': sites_coords,
           'profile': 'foot-walking',
           'metrics': ['duration']}

sites_matrix = ors.distance_matrix(**request)
print("Calculated {}x{} routes.".format(len(sites_matrix['durations']), len(sites_matrix['durations'][0])))

Calculated 9x9 routes.


Check, 26x26. So, we got the durations now in `sites_matrix['durations']`.
Then there's finally the great entrance of [**ortools**](https://github.com/google/or-tools).

Note, this is a local search.

In [9]:
from ortools.constraint_solver import pywrapcp

tsp_size = len(sites_addresses)
num_routes = 1
start = 0  # arbitrary start location

optimal_coords = []

if tsp_size > 0:

    # Old Stuff kept for reference
    # routing = pywrapcp.RoutingModel(tsp_size, num_routes, start)

    # New Way according to ortools v7.0 docs (https://developers.google.com/optimization/support/release_notes#announcing-the-release-of-or-tools-v70)
    # manager = pywrapcp.RoutingIndexManager(num_locations, num_vehicles, depot)
    # routing = pywrapcp.RoutingModel(manager)

    # Adaption according to old and new way
    manager = pywrapcp.RoutingIndexManager(tsp_size, num_routes, start)
    routing = pywrapcp.RoutingModel(manager)


    # Create the distance callback, which takes two arguments (the from and to node indices)
    # and returns the distance between these nodes.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(sites_matrix['durations'][from_node][to_node])


    # Since v7.0, this also needs to be wrapped:
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    # Solve, returns a solution if any.
    assignment = routing.Solve()
    if assignment:
        # Total cost of the 'optimal' solution.
        print("Total duration: " + str(round(assignment.ObjectiveValue(), 3) / 60) + " minutes\n")
        index = routing.Start(start)  # Index of the variable for the starting node.
        route = ''
        # while not routing.IsEnd(index):
        for node in range(routing.nodes()):
            # IndexToNode has been moved from the RoutingModel to the RoutingIndexManager
            optimal_coords.append(sites_coords[manager.IndexToNode(index)])
            route += str(sites_addresses[manager.IndexToNode(index)]) + ' -> '
            index = assignment.Value(routing.NextVar(index))
        route += str(sites_addresses[manager.IndexToNode(index)])
        optimal_coords.append(sites_coords[manager.IndexToNode(index)])
        print("Route:\n" + route)

Total duration: 19.716666666666665 minutes

Route:
Sukiennice -> Kościół pw. Świętego Wojciecha Biskupa i Męczennika -> Kościół pw. Świętej Trójcy ->  -> Hotel Gródek -> Kościół pw. Świętego Tomasza Apostoła -> Kościół pw. Świętej Barbary -> Kościół pw. Wniebowzięcia Najświętszej Marii Panny -> Pomnik Żaka -> Sukiennice


Visualizing the optimal route

In [14]:
def style_function(color):
    return lambda feature: dict(color=color,
                                weight=5,
                                opacity=1)

# And now the optimal route
request = {'coordinates': optimal_coords,
           'profile': 'foot-walking',
           'geometry': 'true',
           'format_out': 'geojson',
           #            'instructions': 'false'
           }
optimal_route = ors.directions(**request)
folium.features.GeoJson(data=optimal_route,
                        name='Your perfect route',
                        style_function=style_function('#27ce49'),
                        overlay=True).add_to(m)

m.add_child(folium.map.LayerControl())
m

In [ ]:
optimal_duration = optimal_route['features'][0]['properties']['summary']['duration'] / 60

print("Duration optimal route: {0:.3f} mins".format(optimal_duration))